In [106]:
import cv2 as cv
import numpy as np
import os

path_fisiere_antrenare = '../Proiect1/CAVA-2024-Tema1/antrenare'
path_folder_cu_raspunsul = './352_Dinu_MateiAlexandru'
dataset_directory = './dataset'

In [107]:
def show_image(title,image):
    image=cv.resize(image,(0,0),fx=0.3,fy=0.3)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [108]:
# show_image("image", cv.imread("../Proiect1/CAVA-2024-Tema1/antrenare/1_01.jpg"))

In [109]:
def select_board_position_and_border_size(image):
    image_original = image.copy()
    image = cv.cvtColor(image,cv.COLOR_BGR2HSV)
    image_m_blur = cv.medianBlur(image,3)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 5) 
    image_sharpened = cv.addWeighted(image_m_blur, 1.2, image_g_blur, -0.8, 0)
    
    masca_initiala = np.array([50, 0, 0])
    masca_finala = np.array([255, 255, 255])
    combined_mask = cv.inRange(image_g_blur, masca_initiala, masca_finala)
    # # show_image('combined_mask',combined_mask)
    
    contours, _ = cv.findContours(combined_mask,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    max_area = 0
    for i in range(len(contours)):
        if(len(contours[i]) > 3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left
                
    width = 2000
    height = 2000
    
    image_copy = image.copy()
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    
    puzzle = np.array([top_left,top_right,bottom_right,bottom_left], dtype = "float32")
    destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")

    M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)

    result = cv.warpPerspective(image, M, (width, height))
    # # show_image("result",result)
    
    return result, cv.warpPerspective(image_original, M, (width, height))

In [110]:
def gaseste_colturile_celui_mai_mare_contur(imagine):
    contours, _ = cv.findContours(imagine,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    ######### # show_image('combined_mask',combined_mask)
    # # show_image('image_thresholded',thresh)
    
    max_area = 0
    for i in range(len(contours)):
        if(len(contours[i]) > 3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left
    return top_left, top_right, bottom_right, bottom_left

In [111]:
def select_inner_grid_position_and_size_in_board(board_image_hsv, board_image_original):
    original_image_copy = board_image_original.copy()
    
    # Save image to local
    cv.imwrite('board_image_original.jpg', board_image_original)
    cv.imwrite('board_image_hsv.jpg', board_image_hsv)
    
    
    
    # Ducem originala in HSV
    board_image_original = cv.cvtColor(board_image_original, cv.COLOR_BGR2HSV)
    masca_initiala = np.array([0, 0, 135])
    masca_finala = np.array([100, 255, 255])
    combined_mask = cv.inRange(board_image_original, masca_initiala, masca_finala)
    # show_image('combined_mask',combined_mask)
    
    
    ##### Aici trebuie sa detectam colturile imaginii
    colt_posibil_dreapta_sus = [(1680, 190), (1790, 320)]
    colt_posibil_dreapta_jos = [(1680, 1690), (1770, 1790)]
    # colt_posibil_stanga_sus = [(190, 190), (320, 350)]
    colt_posibil_stanga_jos = [(200, 1680), (310, 1790)]
   
   
    cv.rectangle(combined_mask, colt_posibil_dreapta_sus[0], colt_posibil_dreapta_sus[1], (0, 0, 255), 2)
    cv.rectangle(combined_mask, colt_posibil_dreapta_jos[0], colt_posibil_dreapta_jos[1], (0, 0, 255), 2)
    cv.rectangle(combined_mask, colt_posibil_stanga_jos[0], colt_posibil_stanga_jos[1], (0, 0, 255), 2)
    # cv.rectangle(combined_mask, colt_posibil_stanga_sus[0], colt_posibil_stanga_sus[1], (0, 0, 255), 2)
    # show_image('combined_mask',combined_mask)
    # Taiem imaginea in functie de colturile detectate
    dreapta_sus = combined_mask[colt_posibil_dreapta_sus[0][1]:colt_posibil_dreapta_sus[1][1], colt_posibil_dreapta_sus[0][0]:colt_posibil_dreapta_sus[1][0]]
    dreapta_jos = combined_mask[colt_posibil_dreapta_jos[0][1]:colt_posibil_dreapta_jos[1][1], colt_posibil_dreapta_jos[0][0]:colt_posibil_dreapta_jos[1][0]]
    # stanga_sus = combined_mask[colt_posibil_stanga_sus[0][1]:colt_posibil_stanga_sus[1][1], colt_posibil_stanga_sus[0][0]:colt_posibil_stanga_sus[1][0]]
    stanga_jos = combined_mask[colt_posibil_stanga_jos[0][1]:colt_posibil_stanga_jos[1][1], colt_posibil_stanga_jos[0][0]:colt_posibil_stanga_jos[1][0]]
    
    # Detectam colturile exacte
    top_left, top_right, bottom_right, bottom_left = gaseste_colturile_celui_mai_mare_contur(dreapta_sus)
    top_left_dreapta_jos, top_right_dreapta_jos, bottom_right_dreapta_jos, bottom_left_dreapta_jos = gaseste_colturile_celui_mai_mare_contur(dreapta_jos)
    # top_left_stanga_sus, top_right_stanga_sus, bottom_right_stanga_sus, bottom_left_stanga_sus = gaseste_colturile_celui_mai_mare_contur(stanga_sus)
    top_left_stanga_jos, top_right_stanga_jos, bottom_right_stanga_jos, bottom_left_stanga_jos = gaseste_colturile_celui_mai_mare_contur(stanga_jos)
    
    nou_punct_topright = tuple(top_right + np.array(colt_posibil_dreapta_sus[0]))
    nou_punct_bottomright = tuple(bottom_right_dreapta_jos + np.array(colt_posibil_dreapta_jos[0]))
    nou_punct_bottomleft = tuple(bottom_left_stanga_jos + np.array(colt_posibil_stanga_jos[0]))
    # Bazat pe cele 3 colturi, calculam al 4-lea colt
    nou_punct_topleft = nou_punct_topright[1], nou_punct_bottomleft[0]
    
    cv.circle(board_image_original,nou_punct_topright,20,(0,0,255),-1)
    cv.circle(board_image_original,nou_punct_bottomright,20,(0,0,255),-1)
    cv.circle(board_image_original,nou_punct_bottomleft,20,(0,0,255),-1)
    cv.circle(board_image_original,nou_punct_topleft,20,(0,0,255),-1)
    
    width = 2000
    height = 2000
    puzzle = np.array([nou_punct_topleft,nou_punct_topright,nou_punct_bottomright,nou_punct_bottomleft], dtype = "float32")
    destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")
    M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)
    result = cv.warpPerspective(original_image_copy, M, (width, height))
    cell_width = width // 14 + 1
    cell_height = height // 14 + 1
    print('cell_width', cell_width)
    print('cell_height', cell_height)
    line_thickness = 5
    for i in range(14):
        cv.line(result, (i * cell_width, 0), (i * cell_width, height), (0, 0, 0), line_thickness)
        cv.line(result, (0, i * cell_height), (width, i * cell_height), (0, 0, 0), line_thickness)
        
    # Impartim imaginea in 14x14
    width, height = combined_mask.shape[:2]
    square_width, square_height = width // 14 + 1, height // 14 + 1
    copie_imagine = result.copy()
    copie_imagine = cv.cvtColor(copie_imagine, cv.COLOR_BGR2HSV)
    masca_imagine_detectie_piese_lipsa_centru_lower = np.array([0, 100, 0])
    masca_imagine_detectie_piese_lipsa_centru_upper = np.array([35, 255, 255])
    masca_imagine_detectie_piese_lipsa_centru = cv.inRange(copie_imagine, masca_imagine_detectie_piese_lipsa_centru_lower, masca_imagine_detectie_piese_lipsa_centru_upper)
    for i in range(14):
        for j in range(14):
            square = masca_imagine_detectie_piese_lipsa_centru[i * square_width:(i + 1) * square_width, j * square_height:(j + 1) * square_height]
            new_square = masca_imagine_detectie_piese_lipsa_centru[i * square_width + 20:(i + 1) * square_width - 20, j * square_height + 20:(j + 1) * square_height - 20]
            if square.size > 0:
                colors, counts = np.unique(square, return_counts=True)
                majoritar = colors[np.argmax(counts)]
                if majoritar == 255:
                    # Make the current square in the result pitch black
                    result[i * square_width:(i + 1) * square_width, j * square_height:(j + 1) * square_height] = 0
    # show_image('result_after_modified_colors',result)
    return result
    
    
    # # print(top_left,top_right,bottom_left,bottom_right)
    # # Extragem imaginea cu tabla de joc
    # width = 2000
    # height = 2000
    
    # image_copy = board_image_original.copy()
    # cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    # cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    # cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    # cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    # # show_image("detected corners",image_copy)
    
    # puzzle = np.array([top_left,top_right,bottom_right,bottom_left], dtype = "float32")
    # destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")

    # M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)

    # result = cv.warpPerspective(board_image_original, M, (width, height))
    # # Divide the board into 14x14 squares
    # square_width = width // 14
    # square_height = height // 14
    # print('square_width',square_width)
    # print('square_height',square_height)
    # # Draw the grid
    # grid_line_thickness = 8
    # # Grid line color is black
    # for i in range(14):
    #     cv.line(result, (i * square_width, 0), (i * square_width, height), (0, 0, 0), grid_line_thickness)
    #     cv.line(result, (0, i * square_height), (width, i * square_height), (0, 0, 0), grid_line_thickness)
    
    # # Write image to local
    # cv.imwrite('resulting_board_item.jpg', result)
    # # show_image("result",result)
    # return result

In [112]:
# select_board_position_and_border_size(cv.imread("../Proiect1/CAVA-2024-Tema1/imagini_auxiliare/04.jpg"))

In [113]:
def altereaza_imaginea_cu_grid_ul(image):
    # show_image('imagine_cu_careul_de_procesat',image)
    cv.imwrite('imagine_cu_careul_de_procesat2.jpg', image)
    
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    masca_pentru_a_vedea_pozitiile_cifrelor_inf = np.array([0, 0, 95])
    masca_pentru_a_vedea_pozitiile_cifrelor_sup = np.array([255, 255, 255])
    # Masca pentru pozitiile cifrelor
    combined_mask = cv.inRange(image, masca_pentru_a_vedea_pozitiile_cifrelor_inf, masca_pentru_a_vedea_pozitiile_cifrelor_sup)
    # show_image('imagine_cu_careul_de_procesat',combined_mask)
    
    pozitii_deja_utilizate_chr = []
    pozitii_deja_utilizate_tuple = []
    width, height = combined_mask.shape[:2]
    square_width, square_height = width // 14 + 1, height // 14 + 1
    for i in range(14):
        for j in range(14):
            square = combined_mask[i * square_width:(i + 1) * square_width, j * square_height:(j + 1) * square_height]
            new_square = combined_mask[i * square_width + 20:(i + 1) * square_width - 20, j * square_height + 20:(j + 1) * square_height - 20]
            cv.rectangle(combined_mask, (j * square_height + 20, i * square_width + 20), ((j + 1) * square_height - 20, (i + 1) * square_width - 20), (0, 255, 0), 2)
            
            mean_intensity = np.mean(new_square)
            if mean_intensity < 240 and mean_intensity > 0: # Era 235 inainte
                combined_mask[i * square_width:(i + 1) * square_width, j * square_height:(j + 1) * square_height] = 100
                pozitii_deja_utilizate_chr.append(str(i + 1) + str(chr(ord('A') + j)))
                pozitii_deja_utilizate_tuple.append((i, j))
                pass
    # show_image('imagine_cu_careul_de_procesat',combined_mask)
    return square_width, square_height, pozitii_deja_utilizate_chr, pozitii_deja_utilizate_tuple
    

In [114]:
def preprocess_digit_image(input_image):
    image_height, image_width = input_image.shape
    # Punem un tresh daca e nevoie, pentru a scoate zgomote
    _, binary_image = cv.threshold(input_image, 127, 255, cv.THRESH_BINARY)
    morphological_kernel = np.ones((3, 3), np.uint8)
    noise_removed_image = cv.morphologyEx(binary_image, cv.MORPH_CLOSE, morphological_kernel, iterations=2)
    cropped_image = noise_removed_image[10:image_height - 10, 10:image_width - 10]
    # # show_image('cropped_image',cropped_image)
    non_zero_pixels = np.where(cropped_image == 255)
    min_vertical, min_horizontal = np.min(non_zero_pixels, axis=1)
    # # print(min_vertical, min_horizontal)
    max_vertical, max_horizontal = np.max(non_zero_pixels, axis=1)
    bounding_box_image = cropped_image[min_vertical:max_vertical, min_horizontal:max_horizontal]
    
    # # show_image('bounding_box_image',bounding_box_image)
    centered_canvas_image = np.zeros((150, 150), dtype=np.uint8)
    canvas_offset_x = (centered_canvas_image.shape[1] - bounding_box_image.shape[1]) // 2
    canvas_offset_y = (centered_canvas_image.shape[0] - bounding_box_image.shape[0]) // 2
    # Centram imaginea in canvas
    centered_canvas_image[canvas_offset_y:canvas_offset_y + bounding_box_image.shape[0], canvas_offset_x:canvas_offset_x + bounding_box_image.shape[1]] = bounding_box_image

    return centered_canvas_image

def create_dataset_from_given_square(square, number_inside_square):
    output_dir = f'{dataset_directory}/digit_{number_inside_square}'
    os.makedirs(output_dir, exist_ok=True)
    ## show_image('square',square)
    processed_square = preprocess_digit_image(square)
    
    # Scriem imaginea in folderul de dataset
    cv.imwrite(f'{output_dir}/digit_{number_inside_square}.jpg', processed_square)
    for angle in range(-6, 7, 2):
        rotation_matrix = cv.getRotationMatrix2D((processed_square.shape[1] // 2, processed_square.shape[0] // 2), angle, 1)
        # Rotire dupa unghi
        rotated_square = cv.warpAffine(processed_square, rotation_matrix, (processed_square.shape[1], processed_square.shape[0]))
        cv.imwrite(f'{output_dir}/digit_{number_inside_square}_rotated_{angle}.jpg', rotated_square)
        for dx in range(-6, 7, 2):
            for dy in range(-6, 7, 2):
                if dx == 0 and dy == 0:
                    continue
                translated_square = np.roll(rotated_square, shift=(dy, dx), axis=(0, 1))
                cv.imwrite(f'{output_dir}/digit_{number_inside_square}_rotated_{angle}_translated_{dx}_{dy}.jpg', translated_square)
    return square


dataset = []
for folder_name in os.listdir(dataset_directory):
    digit_label = int(folder_name.split('_')[1])
    folder_path = os.path.join(dataset_directory, folder_name)
    for filename in os.listdir(folder_path):
        print("Reading image", filename)
        image_path = os.path.join(folder_path, filename)
        grayscale_image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
        dataset.append((grayscale_image, digit_label))


def classify_digit(digit_image):
    print(f"Calculating distances for {len(dataset)} images")
    distances = []
    for dataset_image, dataset_digit in dataset:
        euclidean_distance = np.linalg.norm(digit_image - dataset_image)
        distances.append((euclidean_distance, dataset_digit))
        
    distances.sort(key=lambda x: x[0])
    #  print(distances)
    # Cele mai apropiate 5 cifre
    top5_nearest_neighbors = [distances[i] for i in range(5)]
    digit_frequencies = {}
    for _, digit in top5_nearest_neighbors:
        # Dict
        digit_frequencies[digit] = digit_frequencies.get(digit, 0) + 1

    predicted_digit = max(digit_frequencies, key=digit_frequencies.get)
    print(f"Predicted digit: {predicted_digit}")
    return predicted_digit, distances


Reading image digit_13_rotated_2_translated_-6_-4.jpg
Reading image digit_13_rotated_-4_translated_2_0.jpg
Reading image digit_13_rotated_0_translated_2_-2.jpg
Reading image digit_13_rotated_4_translated_2_4.jpg
Reading image digit_13_rotated_2_translated_2_0.jpg
Reading image digit_13_rotated_-4_translated_0_-6.jpg
Reading image digit_13_rotated_-6_translated_0_-4.jpg
Reading image digit_13_rotated_2_translated_4_0.jpg
Reading image digit_13_rotated_-2_translated_-6_4.jpg
Reading image digit_13_rotated_6_translated_-4_-2.jpg
Reading image digit_13_rotated_-4_translated_-4_4.jpg
Reading image digit_13_rotated_-4_translated_2_-6.jpg
Reading image digit_13_rotated_-6_translated_0_-6.jpg
Reading image digit_13_rotated_2_translated_-4_2.jpg
Reading image digit_13_rotated_6_translated_2_0.jpg
Reading image digit_13_rotated_-4_translated_-6_-2.jpg
Reading image digit_13_rotated_6_translated_2_4.jpg
Reading image digit_13_rotated_2_translated_-6_2.jpg
Reading image digit_13_rotated_4_translat

In [115]:
def clasifica_fiecare_celula_din_imagine(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    masca_pentru_a_vedea_pozitiile_cifrelor_inf = np.array([0, 0, 95])
    masca_pentru_a_vedea_pozitiile_cifrelor_sup = np.array([255, 255, 255])
    # Aplicam masca pentru a vedea pozitiile cifrelor
    combined_mask = cv.inRange(image, masca_pentru_a_vedea_pozitiile_cifrelor_inf, masca_pentru_a_vedea_pozitiile_cifrelor_sup)
    # show_image('imagine_cu_careul_de_procesat',combined_mask)
    
    pozitii_deja_utilizate = []
    # Impartim imaginea in 14x14
    width, height = combined_mask.shape[:2]
    square_width, square_height = width // 14 + 1, height // 14 + 1
    # Pentru fiecare patrat, calculam media intensitatii
    for i in range(14):
        for j in range(14):
            square = combined_mask[i * square_width:(i + 1) * square_width, j * square_height:(j + 1) * square_height]
            new_square = combined_mask[i * square_width + 20:(i + 1) * square_width - 20, j * square_height + 20:(j + 1) * square_height - 20]
            # cvs.rectangle(combined_mask, (j * square_height + 20, i * square_width + 20), ((j + 1) * square_height - 20, (i + 1) * square_width - 20), (0, 255, 0), 2)
            mean_intensity = np.mean(new_square)
            # Daca media intensitatii este <100, atunci este un patrat negru
            if mean_intensity < 240 and mean_intensity > 0:
                new_square = combined_mask[i * square_width + 10:(i + 1) * square_width - 10, j * square_height + 10:(j + 1) * square_height - 10]
                # cv.rectangle(combined_mask, (j * square_height + 10, i * square_width + 10), ((j + 1) * square_height - 10, (i + 1) * square_width - 10), (0, 255, 0), 2)
                inverted_square = 255 - new_square
                inverted_square[inverted_square > 0] = 255
                # Inverseaza culorile
                altered_square = preprocess_digit_image(inverted_square)
                # show_image('inverted_square',altered_square)/
                
                cifra, distante_imagini_sortate = classify_digit(altered_square)
                
                # Avem un patrat plin, il coloram cu grey
                combined_mask[i * square_width:(i + 1) * square_width, j * square_height:(j + 1) * square_height] = 100
                
                pozitii_deja_utilizate.append((mean_intensity, i, j, cifra))
    return pozitii_deja_utilizate, combined_mask
    

In [116]:

def clasifica_doar_o_celula(celula_din_imagine):
    image = cv.cvtColor(celula_din_imagine, cv.COLOR_BGR2HSV)
    masca_pentru_a_vedea_pozitiile_cifrelor_inf = np.array([0, 0, 95])
    masca_pentru_a_vedea_pozitiile_cifrelor_sup = np.array([255, 255, 255])
    # Aplicam masca pentru a vedea pozitiile cifrelor
    combined_mask = cv.inRange(image, masca_pentru_a_vedea_pozitiile_cifrelor_inf, masca_pentru_a_vedea_pozitiile_cifrelor_sup)
    
    # Un nou patrat
    square_width, square_height = combined_mask.shape
    new_square = combined_mask[10:square_width - 10, 10:square_height - 10]
    inverted_square = 255 - new_square
    inverted_square[inverted_square > 0] = 255
    # Procesam imaginea curenta
    altered_square = preprocess_digit_image(inverted_square)
    cifra, distante_imagini_sortate = classify_digit(altered_square)
    return cifra, distante_imagini_sortate

In [117]:

import os
lista_curenta_pozitii = {} # Pentru fiecare runda, retinem o lista
files = []
lista_raspuns_final_pozitii_numere = []

os.makedirs('test', exist_ok=True)
for imagine_path in os.listdir(path_fisiere_antrenare):
    # If it is a jpg file
    if not imagine_path.endswith('.jpg'):
        continue
    files.append(imagine_path.split('.')[0])

# Sortam fisierele
files.sort(key=lambda x: (int(x.split('_')[0]), int(x.split('_')[1])))

for imagine_path in files:
    # Cream key-ul pentru runda curenta
    if imagine_path.split('_')[0] not in lista_curenta_pozitii:
        lista_curenta_pozitii[imagine_path.split('_')[0]] = []
        
    print("Analizam imaginea", imagine_path)
    modified, orig = select_board_position_and_border_size(cv.imread(f'{path_fisiere_antrenare}/{imagine_path}.jpg'))
    grid_intern_de_procesat = select_inner_grid_position_and_size_in_board(modified, orig)
    sqwidth, sqheight, pozitii_chr, pozitii_tuple = altereaza_imaginea_cu_grid_ul(grid_intern_de_procesat)
    print(pozitii_tuple)
    
    # Adaugam la lista de pozitii pozitiile curente, daca nu sunt deja adaugate
    for pozitie, (y, x) in zip(pozitii_chr, pozitii_tuple):
        if pozitie not in lista_curenta_pozitii[imagine_path.split('_')[0]]:
            lista_curenta_pozitii[imagine_path.split('_')[0]].append(pozitie)
            print(pozitie, x, y)
            
            celula = grid_intern_de_procesat[y * sqwidth:(y + 1) * sqwidth, x * sqheight:(x + 1) * sqheight]
            
            # show_image('celula',celula)
            clasificare_celula, distante_imagini_sortate = clasifica_doar_o_celula(celula)
            
            # Creare folder
            os.makedirs(f'{path_folder_cu_raspunsul}/', exist_ok=True)
            with open(f'{path_folder_cu_raspunsul}/{imagine_path}.txt', 'a') as f:
                f.write(pozitie + " " + f"{clasificare_celula}" + '\n')
            
            lista_raspuns_final_pozitii_numere.append(((y, x), clasificare_celula, distante_imagini_sortate))
            
            break


Analizam imaginea 1_01
cell_width 143
cell_height 143
[(8, 7)]
9H 7 8
Calculating distances for 11638 images
Predicted digit: 8
Analizam imaginea 1_02
cell_width 143
cell_height 143
[(8, 7), (9, 7)]
10H 7 9
Calculating distances for 11638 images
Predicted digit: 32
Analizam imaginea 1_03
cell_width 143
cell_height 143
[(8, 7), (9, 7), (10, 7)]
11H 7 10
Calculating distances for 11638 images
Predicted digit: 45
Analizam imaginea 1_04
cell_width 143
cell_height 143
[(8, 7), (9, 7), (10, 7), (11, 7)]
12H 7 11
Calculating distances for 11638 images
Predicted digit: 72
Analizam imaginea 1_05
cell_width 143
cell_height 143
[(5, 7), (8, 7), (9, 7), (10, 7), (11, 7)]
6H 7 5
Calculating distances for 11638 images
Predicted digit: 8
Analizam imaginea 1_06
cell_width 143
cell_height 143
[(4, 7), (5, 7), (8, 7), (9, 7), (10, 7), (11, 7)]
5H 7 4
Calculating distances for 11638 images
Predicted digit: 10
Analizam imaginea 1_07
cell_width 143
cell_height 143
[(3, 7), (4, 7), (5, 7), (8, 7), (9, 7), (

In [118]:
import copy
board_configuration = [
    ["x3", -1, -1, -1, -1, -1, "x3", "x3", -1, -1, -1, -1, -1, "x3"], 
    [-1, "x2", -1, -1, "/", -1, -1, -1, -1, "/", -1, -1, "x2", -1], 
    [-1, -1, "x2", -1, -1, "-", -1, -1, "-", -1, -1, "x2", -1, -1], 
    [-1, -1, -1, "x2", -1, -1, "+", "x", -1, -1, "x2", -1, -1, -1], 
    [-1, "/", -1, -1, "x2", -1, "x", "+", -1, "x2", -1, -1, "/", -1], 
    [-1, -1, "-", -1, -1, -1, -1, -1, -1, -1, -1, "-", -1, -1], 
    ["x3", -1, -1, "x", "+", -1, 1, 2, -1, "x", "+", -1, -1, "x3"], 
    ["x3", -1, -1, "+", "x", -1, 3, 4, -1, "+", "x", -1, -1, "x3"], 
    [-1, -1, "-", -1, -1, -1, -1, -1, -1, -1, -1, "-", -1, -1], 
    [-1, "/", -1, -1, "x2", -1, "+", "x", -1, "x2", -1, -1, "/", -1], 
    [-1, -1, -1, "x2", -1, -1, "x", "+", -1, -1, "x2", -1, -1, -1], 
    [-1, -1, "x2", -1, -1, "-", -1, -1, "-", -1, -1, "x2", -1, -1], 
    [-1, "x2", -1, -1, "/", -1, -1, -1, -1, "/", -1, -1, "x2", -1], 
    ["x3", -1, -1, -1, -1, -1, "x3", "x3", -1, -1, -1, -1, -1, "x3"]
]
initial_board_configuration = copy.deepcopy(board_configuration)

def display_game_board(game_board, recently_added_piece_position=None):
    def format_board_cell(cell_value, is_newly_added_cell=False):
        displayed_cell_content = str(cell_value) if cell_value != -1 else " "
        if is_newly_added_cell:
            return f"[{displayed_cell_content}]"
        else:
            return f" {displayed_cell_content} "
    
    print("\n    " + " ".join(f"{column_index:>2}" for column_index in range(len(game_board[0]))))
    print("   " + "-" * (len(game_board[0]) * 4))
    
    for row_index, row_contents in enumerate(game_board):
        row_display = f"{row_index:>2} |" + "".join(
            format_board_cell(
                cell_value, 
                recently_added_piece_position == (row_index, column_index)
            ) for column_index, cell_value in enumerate(row_contents)
        )
        print(row_display)
        
def solveMathableBoardConfiguration(added_piece_position, added_piece_value):
    # Unpack the row and column from the added piece position
    row, column = added_piece_position

    def is_valid_piece(piece):
        return piece == -1 or (isinstance(piece, int) and piece != -1)
    
    def is_valid_equation_position(r, c):
        forbidden_symbols = ["x3", "x2", "/", "-", "+"]
        return board_configuration[r][c] not in forbidden_symbols and is_valid_piece(board_configuration[r][c])

    def check_equation_in_direction(direction_pieces, operator, target_value):
        piece_a, piece_b = direction_pieces
        if piece_a == -1 or piece_b == -1:
            return 0

        is_valid_equation = False  
        if operator == "+":
            addition_result = piece_a + piece_b
            if addition_result == target_value:
                is_valid_equation = True

        elif operator == "-":
            subtraction_result = abs(piece_a - piece_b)
            if subtraction_result == target_value:
                is_valid_equation = True
        elif operator == "x":
            multiplication_result = piece_a * piece_b
            if multiplication_result == target_value:
                is_valid_equation = True
        
        elif operator == "/":
            if piece_b != 0:
                division_result_ab = piece_a / piece_b
                if division_result_ab == target_value:
                    is_valid_equation = True
            if piece_a != 0:
                division_result_ba = piece_b / piece_a
                if division_result_ba == target_value:
                    is_valid_equation = True
        elif operator == "all":
            addition_result = piece_a + piece_b
            subtraction_result = abs(piece_a - piece_b)
            multiplication_result = piece_a * piece_b
            division_result_ab = piece_a / piece_b if piece_b != 0 else None
            division_result_ba = piece_b / piece_a if piece_a != 0 else None

            if (addition_result == target_value or 
                subtraction_result == target_value or 
                multiplication_result == target_value or 
                (division_result_ab == target_value) or 
                (division_result_ba == target_value)):
                is_valid_equation = True
        return 1 if is_valid_equation else 0

    def check_equation_in_specific_direction(start_row, start_col, row_offset, col_offset, board, operator, target_value):
        ## Offseturile
        first_piece_row = start_row + row_offset
        first_piece_col = start_col + col_offset
        second_piece_row = start_row + 2 * row_offset
        second_piece_col = start_col + 2 * col_offset
        if (0 <= first_piece_row < len(board) and 0 <= first_piece_col < len(board[0]) and 0 <= second_piece_row < len(board) and 0 <= second_piece_col < len(board[0])):
            if is_valid_equation_position(first_piece_row, first_piece_col) and is_valid_equation_position(second_piece_row, second_piece_col):
                # Daca piesele sunt valide, verificam ecuatia
                direction_pieces = (board[first_piece_row][first_piece_col], board[second_piece_row][second_piece_col])
                return check_equation_in_direction(direction_pieces, operator, target_value)
        return 0

    current_operator = board_configuration[row][column]
    if current_operator in ["+", "-", "x", "/"]:
        operator = current_operator
    else:
        operator = "all"
    up_equations = check_equation_in_specific_direction(row, column, -1, 0, board_configuration, operator, added_piece_value)
    down_equations = check_equation_in_specific_direction(row, column, 1, 0, board_configuration, operator, added_piece_value)
    left_equations = check_equation_in_specific_direction(row, column, 0, -1, board_configuration,operator, added_piece_value)
    right_equations = check_equation_in_specific_direction(row, column, 0, 1, board_configuration, operator, added_piece_value)
    # Totale
    total_valid_equations = up_equations + down_equations + left_equations + right_equations
    
    if board_configuration[row][column] == "x3":
        return 3 * total_valid_equations
    elif board_configuration[row][column] == "x2":
        return 2 * total_valid_equations
    else:
        return total_valid_equations

def modificaRezultatTest(jocCurent, indexInJOC, pozitie, cifra):
    if indexInJOC < 10:
        os.remove(f'{path_folder_cu_raspunsul}/{jocCurent}_0{indexInJOC}.txt')
    else:
        os.remove(f'{path_folder_cu_raspunsul}/{jocCurent}_{indexInJOC}.txt')
    pozitie = str(pozitie[0] + 1) + str(chr(ord('A') + pozitie[1]))
    if indexInJOC < 10:
        with open(f'{path_folder_cu_raspunsul}/{jocCurent}_0{indexInJOC}.txt', 'a') as f:
            f.write(pozitie + " " + f"{cifra}" + '\n')
    else:
        with open(f'{path_folder_cu_raspunsul}/{jocCurent}_{indexInJOC}.txt', 'a') as f:
            f.write(pozitie + " " + f"{cifra}" + '\n')
            
            
jocul_curent = 1
index_joc = 1
scor_acumulat = 0
player_turns = {}
player_scores = {}
index_joc_anterior_pentru_scoruri = None
player_anterior_pentru_scoruri = None
#Incarca din turns
def load_player_turns(joc):
    player_turns.clear()
    player_turns['Player1'] = []
    player_turns['Player2'] = []
    
    try:
        with open(f'{path_fisiere_antrenare}/{joc}_turns.txt', 'r') as f:
            for linie_fis in f:
                lista_loc = linie_fis.strip().split()
                if len(lista_loc) == 2:
                    player, turn = lista_loc
                    player_turns[player].append(int(turn))
        print(f"Player turns for game {joc} loaded successfully!")
        print("Player turns:", player_turns)
    except FileNotFoundError:
        print(f"Turns file for game {joc} not found!")
    except Exception as e:
        print(f"Error reading turns file: {e}")

def is_player_turn(player, current_turn):
    return current_turn in player_turns[player]
# Salvarea scorurilor
def save_player_scores(joc):
    try:
        with open(f'{path_folder_cu_raspunsul}/{joc}_scores.txt', 'w') as f:
            all_turns = []
            for player in ['Player1', 'Player2']:
                for turn in player_turns[player]:
                    score = player_scores.get((player, turn), 0)
                    all_turns.append((turn, player, score))
                    
            all_turns.sort(key=lambda x: x[0])
            for turn, player, score in all_turns:
                f.write(f"{player} {turn} {score}\n")
    except Exception as e:
        print(f"Error saving scores for game {joc}: {e}")

# Pentru fiecare pozitie si clasificare
for pozitie, clasificare_celula, probabilitati_celula in lista_raspuns_final_pozitii_numere:
    # Incarca player turns
    if index_joc == 1:
        player_scores.clear()
        load_player_turns(jocul_curent)
    
    print("############ JOCUL CURENT ESTE ############", jocul_curent, " Iar indexul jocului este ", index_joc)
    print(pozitie, clasificare_celula)
    
    # Daca configuratia este invalida
    if solveMathableBoardConfiguration(pozitie, clasificare_celula) == 0:
        print("The board configuration is invalid!")
        for distanta, cifra in probabilitati_celula:
            # Pana cand este valida
            if solveMathableBoardConfiguration(pozitie, cifra) != 0:
                print(f"Setam tabla de la pozitia {pozitie} cu cifra {cifra}")
                current_score = solveMathableBoardConfiguration(pozitie, cifra)
                current_score *= cifra
                # Actualizam
                for player in ['Player1', 'Player2']:
                    if is_player_turn(player, index_joc):
                        print(f"La {index_joc} este randul lui {player}")
                        if index_joc_anterior_pentru_scoruri and player_anterior_pentru_scoruri:
                            player_scores[(player_anterior_pentru_scoruri, index_joc_anterior_pentru_scoruri)] = player_scores.get((player_anterior_pentru_scoruri, index_joc_anterior_pentru_scoruri), 0) + scor_acumulat
                            index_joc_anterior_pentru_scoruri = index_joc
                            player_anterior_pentru_scoruri = player
                            scor_acumulat = 0
                        else:
                            index_joc_anterior_pentru_scoruri = index_joc
                            player_anterior_pentru_scoruri = player
                        break
                
                scor_acumulat += current_score
                print("Current score is", current_score)
                print("Scor acumulat este", scor_acumulat)
                
                modificaRezultatTest(jocul_curent, index_joc, pozitie, cifra)
                board_configuration[pozitie[0]][pozitie[1]] = cifra
                display_game_board(board_configuration, pozitie)
                break
            else:
                print(f"Nu putem seta tabla de la pozitia {pozitie} cu cifra {cifra}")
    else:
        #Doar actualizare
        current_score = solveMathableBoardConfiguration(pozitie, clasificare_celula) 
        current_score *= clasificare_celula
        for player in ['Player1', 'Player2']:
            if is_player_turn(player, index_joc):        
                print(f"La {index_joc} este randul lui {player}")
                if index_joc_anterior_pentru_scoruri and player_anterior_pentru_scoruri:
                    player_scores[(player_anterior_pentru_scoruri, index_joc_anterior_pentru_scoruri)] = player_scores.get((player_anterior_pentru_scoruri, index_joc_anterior_pentru_scoruri), 0) + scor_acumulat
                    index_joc_anterior_pentru_scoruri = index_joc
                    player_anterior_pentru_scoruri = player
                    scor_acumulat = 0
                else:
                    index_joc_anterior_pentru_scoruri = index_joc
                    player_anterior_pentru_scoruri = player
                break
        scor_acumulat += current_score
        print("Current score is", current_score)
        print("Scor acumulat este", scor_acumulat)
        board_configuration[pozitie[0]][pozitie[1]] = clasificare_celula
        display_game_board(board_configuration, pozitie)
    
    index_joc += 1
    if index_joc == 51:
        # Salvam in fisier
        if scor_acumulat and index_joc_anterior_pentru_scoruri and player_anterior_pentru_scoruri:
            player_scores[(player_anterior_pentru_scoruri, index_joc_anterior_pentru_scoruri)] = player_scores.get((player_anterior_pentru_scoruri, index_joc_anterior_pentru_scoruri), 0) + scor_acumulat
            scor_acumulat = 0
            
        save_player_scores(jocul_curent)
        print("############ RESETAM MATRICEA DE CONFIGURATIE A TABLEI ############")
        board_configuration = copy.deepcopy(initial_board_configuration)
        jocul_curent += 1
        index_joc = 1
    

Player turns for game 1 loaded successfully!
Player turns: {'Player1': [1, 11, 18, 24, 31, 36, 43], 'Player2': [5, 14, 21, 28, 34, 39, 48]}
############ JOCUL CURENT ESTE ############ 1  Iar indexul jocului este  1
(8, 7) 8
La 1 este randul lui Player1
Current score is 8
Scor acumulat este 8

     0  1  2  3  4  5  6  7  8  9 10 11 12 13
   --------------------------------------------------------
 0 | x3                 x3  x3                 x3 
 1 |    x2        /              /        x2    
 2 |       x2        -        -        x2       
 3 |          x2        +  x        x2          
 4 |    /        x2     x  +     x2        /    
 5 |       -                          -       
 6 | x3        x  +     1  2     x  +        x3 
 7 | x3        +  x     3  4     +  x        x3 
 8 |       -             [8]          -       
 9 |    /        x2     +  x     x2        /    
10 |          x2        x  +        x2          
11 |       x2        -        -        x2       
12 |    x2    

In [119]:
# modified, orig = select_board_position_and_border_size(cv.imread("../Proiect1/CAVA-2024-Tema1/imagini_auxiliare/04.jpg"))
# # modified, orig = select_board_position_and_border_size(cv.imread("../Proiect1/CAVA-2024-Tema1/evaluare/fake_test/1_01.jpg"))

# grid_intern_de_procesat = select_inner_grid_position_and_size_in_board(modified, orig)
# # Extragem patratele din grid
# grid_w, grid_h = grid_intern_de_procesat.shape[:2]
# square_w, square_h = grid_w // 14, grid_h // 14
# pozitii, masca = creaza_dataset_de_train(grid_intern_de_procesat)
# print(f'Pozitii: {pozitii}')

